## Timeseries demo
<br>
There's two ways to extract a time series: for a point or for a polygon. There's an example for both below. <br>
If you prefer a GUI, then check out [https://viewer.terrascope.be/], and go to <b>Area of Interest</b>. <br>
When you use this Notebook, you have access to more layers than in the GUI.


In [1]:
import datetime
import requests
import matplotlib.pyplot as plt
import numpy as np
tsvBaseURL='https://services.terrascope.be/timeseries/v1.0/ts/'

response = requests.get(tsvBaseURL) #this returns the layers that are available



In [2]:
if response.status_code == 200:
    layerlist = response.json()['layers']
else:
    raise IOError(response.text)


let's investigate the layers that are offered. The list is not sorted, so let's sort it first.

In [3]:
layersSorted = sorted(layerlist, key=lambda k: k['name']) 

In [4]:
count=0
for l in layersSorted:
    if len(l['dates'])>0:
        count=count+1
        print(l['name'].ljust(40), '{0:4d} '.format(len(l['dates'])), min(l['dates'])[:19], 'to', max(l['dates'])[:19])
print(">>>", count, 'layers with data')
print('Layers with no dates')
count=0
for l in layersSorted:
    if len(l['dates'])==0:
        count=count+1
        print(l['name'])
print(">>>", count, 'layers without data')


BIOPAR_FAPAR300_V1_GLOBAL                 129  2013-10-16T00:00:00 to 2017-05-04T00:00:00
CGLS_LC100_COV_LCCS                      1614  2015-07-04T00:00:00 to 2020-08-24T00:00:00
CHIRPS_RAINFALL                           792  1998-01-11T00:00:00 to 2020-07-21T00:00:00
DCS4COP_CHL_V103                          799  2016-01-02T10:54:42 to 2020-05-30T12:27:01
DCS4COP_CLOUDCOVER_V103                   810  2016-01-02T10:54:42 to 2020-05-30T12:27:01
DCS4COP_FLANDERS_CHL                      454  2017-01-02T11:14:42 to 2018-10-31T11:02:01
DCS4COP_FLANDERS_CHL_V100                 750  2017-01-02T11:14:42 to 2020-02-10T10:52:01
DCS4COP_FLANDERS_CHL_V102                 890  2017-01-02T11:14:42 to 2020-06-02T10:56:31
DCS4COP_FLANDERS_CLOUDCOVER               454  2017-01-02T11:14:42 to 2018-10-31T11:02:01
DCS4COP_FLANDERS_CLOUDCOVER_L8            159  2017-01-03T10:40:38 to 2018-08-27T10:33:22
DCS4COP_FLANDERS_CLOUDCOVER_L8_V100        38  2019-05-10T10:30:59 to 2019-05-31T10:53:42
DCS4COP_FL

Let's first extract the timeseries of a point, for all layers that have 'TERRASCOPE' in their name

In [5]:
TSlayers = []
for l in layersSorted:
    if 'TERRASCOPE' in l['name']:
        TSlayers.append(l['name'])
TSlayers

['TERRASCOPE_S1_SLC_COHERENCE_V1_VH',
 'TERRASCOPE_S1_SLC_COHERENCE_V1_VV',
 'TERRASCOPE_S2_FAPAR_V2',
 'TERRASCOPE_S2_FCOVER_V2',
 'TERRASCOPE_S2_LAI_V2',
 'TERRASCOPE_S2_NDVI_V2']

In [6]:
def getTimeseriesForPoint(covId,tsvBaseURL='https://services.terrascope.be/timeseries/v1.0/ts/',
                          start=datetime.date(2016,1,1),end=datetime.date(2030,12,31),
                          lat=51.146,lon=3.682,printURL=False):

    tsURL = tsvBaseURL + covId + '/point'
    payload = {
            'lon': str(lon),
            'lat': str(lat),
            'startDate': start.strftime('%Y-%m-%d'),
            'endDate': end.strftime('%Y-%m-%d')
    }
    if printURL:
        print(tsURL,payload)
    response=requests.get(tsURL,params=payload)
    if response.status_code == 200:
        timeseries = response.json()['results']
        return(timeseries)
    else:
        return([])
    
    

In [7]:
print('layer'.ljust(40), 'points')
print('---------------------------------------  ------')
for l in TSlayers:
    ts = getTimeseriesForPoint(covId=l,start=datetime.date(2020,1,1), end = datetime.date(2020,7,1), printURL=False)
    print(l.ljust(40),'{0:6d}'.format(len(ts)))


layer                                    points
---------------------------------------  ------
TERRASCOPE_S1_SLC_COHERENCE_V1_VH             0
TERRASCOPE_S1_SLC_COHERENCE_V1_VV             0
TERRASCOPE_S2_FAPAR_V2                       97
TERRASCOPE_S2_FCOVER_V2                      97
TERRASCOPE_S2_LAI_V2                         97
TERRASCOPE_S2_NDVI_V2                        97


So, there are no results for coherence. This needs further analysis.<br>
<br>
Let's have a look at the last time series that we have extracted, for NDVI.<br>
We'll only list the points that are valid (all others cannot be determined due to cloud cover.

In [8]:
print('date        average')
print('----------  -------')
for d in ts:
    if d['result']['validCount']>0:
        print(d['date'], '  {0:.4f}'.format(d['result']['average']))

date        average
----------  -------
2020-01-01   0.2360
2020-01-04   0.3240
2020-01-06   0.3200
2020-01-19   0.3640
2020-01-26   0.3320
2020-02-13   0.4080
2020-03-19   0.4880
2020-03-21   0.5160
2020-03-24   0.5080
2020-03-26   0.5080
2020-03-31   0.5000
2020-04-05   0.5600
2020-04-08   0.6000
2020-04-10   0.5960
2020-04-13   0.7040
2020-04-20   0.7760
2020-04-23   0.7960
2020-04-25   0.7840
2020-05-05   0.8800
2020-05-15   0.8760
2020-05-18   0.8800
2020-05-20   0.8280
2020-05-23   0.8520
2020-05-25   0.8800
2020-05-28   0.9160
2020-05-30   0.8800
2020-06-07   0.8480
2020-06-22   0.6080
2020-06-24   0.8760


On to time series for a polygon then. <br>
This is done using a POST request, rather than a GET request for a point.

In [16]:
def getTimeseriesForPolygon(covId, polylist,
                            tsvBaseURL='https://services.terrascope.be/timeseries/v1.0/ts/',
                            start=datetime.date(2016,1,1),end=datetime.date(2030,12,31),
                            printURL=False):


    tsURL = tsvBaseURL + covId 
    # print(tsvURL)
    tsURL = tsURL +'/geometry'

    payload = {
            "type": "Feature",
            "geometry": {
            "type": "Polygon",
            "coordinates": [
            
                polylist
            
            ]
            }
        }


#    payload['startDate'] = start.strftime('%Y-%m-%d')
#    payload['endDate'] = end.strftime('%Y-%m-%d')
    if printURL:
        print(tsURL, payload)
    response=requests.post(url=tsURL,json=payload, params={'startDate':start.strftime('%Y-%m-%d'), 'endDate':end.strftime('%Y-%m-%d')})

    if response.status_code==200:
        timeSeries = response.json()['results']   
        return(timeSeries)
    else:
        return([])

In [13]:
polygon=[]

point = [3.65, 51.14]
polygon.append(point)
point = [3.66, 51.14]
polygon.append(point)
point = [3.66, 51.15]
polygon.append(point)
point = [3.65, 51.15]
polygon.append(point)
point = [3.65, 51.14]
polygon.append(point)


In [18]:
PolyResults=[]
for l in TSlayers:
    Tresult={}
    Tresult['name']=l
    Tresult['series']= getTimeseriesForPolygon(covId=l,polylist=polygon, start = datetime.date(2020,1,1), end = datetime.date(2020,5,1),printURL=False)
    PolyResults.append(Tresult)
    print(Tresult['name'], len(Tresult['series']))
    

TERRASCOPE_S1_SLC_COHERENCE_V1_VH 86
TERRASCOPE_S1_SLC_COHERENCE_V1_VV 86
TERRASCOPE_S2_FAPAR_V2 64
TERRASCOPE_S2_FCOVER_V2 64
TERRASCOPE_S2_LAI_V2 64
TERRASCOPE_S2_NDVI_V2 64


Okay, let's print the time series.<Br>

In [19]:
ind=3
print(PolyResults[ind]['name'])
print('date'.ljust(11), 'total', ' valid', '  ratio','average')
print('---------- ------ ------  ------ -------')
cutoff=0.9


for r in PolyResults[ind]['series']:
    if r['result']['validCount']/r['result']['totalCount'] > cutoff:
        print(r['date'], '{0:6d}'.format(r['result']['totalCount']), '{0:6d}'.format(r['result']['validCount']), 
              ' {0:.4f}'.format(r['result']['validCount']/r['result']['totalCount']), ' {0:.4f}'.format(r['result']['average']))


TERRASCOPE_S2_FCOVER_V2
date        total  valid   ratio average
---------- ------ ------  ------ -------
2020-01-04  21576  21142  0.9799  0.2861
2020-01-06  21576  21098  0.9778  0.3124
2020-01-19  21576  21418  0.9927  0.3382
2020-01-26  21576  21458  0.9945  0.2457
2020-03-19  21576  21338  0.9890  0.3927
2020-03-21  21576  21368  0.9904  0.4822
2020-03-24  21576  21576  1.0000  0.4615
2020-03-26  21576  21576  1.0000  0.4331
2020-03-31  21576  21377  0.9908  0.4588
2020-04-05  21576  21576  1.0000  0.4650
2020-04-08  21576  21576  1.0000  0.4645
2020-04-10  21576  21576  1.0000  0.4592
2020-04-13  21576  20333  0.9424  0.4772
2020-04-20  21576  21576  1.0000  0.5071
2020-04-23  21576  21474  0.9953  0.5232
2020-04-25  21576  20585  0.9541  0.5155


This is okay: 
- the total number is the number of pixels in the polygon; it is constant, because the polygon doesn't change.
- the valid number is the number of pixels that are not covered by clouds. We are just printing the measures that have 90% good pixels
- the ratio is valid/total
- average is the value of the indicator at that date averaged over all pixels
<br>
No, let's have a look at the coherence time series

In [20]:
ind=1
print(PolyResults[ind]['name'])
print('date'.ljust(11), 'total', ' valid', '  ratio','average')
print('---------- ------ ------  ------ -------')
cutoff=0.9

for r in PolyResults[ind]['series']:
    if r['result']['validCount']/r['result']['totalCount'] > cutoff:
        print(r['date'], '{0:6d}'.format(r['result']['totalCount']), '{0:6d}'.format(r['result']['validCount']), 
              ' {0:.4f}'.format(r['result']['validCount']/r['result']['totalCount']), ' {0:.4f}'.format(r['result']['average']))


TERRASCOPE_S1_SLC_COHERENCE_V1_VV
date        total  valid   ratio average
---------- ------ ------  ------ -------
2020-01-03  86304  86304  1.0000  0.6427
2020-01-05 194184 186012  0.9579  0.5306
2020-01-09  86304  86304  1.0000  0.5806
2020-01-11 194184 194184  1.0000  0.6472
2020-01-15  86304  86304  1.0000  0.5739
2020-01-18 107880 107880  1.0000  0.6755
2020-01-21  86304  86304  1.0000  0.4931
2020-01-23 194184 194184  1.0000  0.6512
2020-01-24 107880 107880  1.0000  0.5771
2020-01-27  86304  86304  1.0000  0.6815
2020-01-30 107880 107880  1.0000  0.5065
2020-02-02  86304  86304  1.0000  0.6408
2020-02-05 107880 107880  1.0000  0.4870
2020-02-08  86304  86304  1.0000  0.6229
2020-02-10 194184 187659  0.9664  0.5236
2020-02-11 107880 107880  1.0000  0.4499
2020-02-14  86304  86304  1.0000  0.5887
2020-02-16 194184 193167  0.9948  0.5098
2020-02-17 107880 107880  1.0000  0.3991
2020-02-20  86304  86304  1.0000  0.6385
2020-02-22 194184 190674  0.9819  0.5490
2020-02-23 107880 10788

This is strange: 
- the total number is the number of pixels in the polygon; it is <b>not</b> constant, probably because there are multiple entries per date in the time series.
- the valid number is <b>not</b> constant and equal to total. As RADAR is cloud penetrating, there's no reason for pixels to be invalid
